##### Import Libs

In [1]:
import pandas as pd
from transformers import pipeline, Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
from datasets import load_dataset
from sklearn.metrics import classification_report

c:\Users\djbac\anaconda3\envs\torch-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### Load the Dataset

In [2]:
# Load the CSV dataset
# Can't specify split because the dataset has not been divided to train and test yet
# So everything is loaded to "train"
dataset = load_dataset('csv', data_files='./datasets/youtoxic_english_1000.csv')

# Define a function to do some preprocessing - rename target and map values
# I'll rename the target column "IsToxic" to "labels".
def preprocess_data(data):
    # Rename columns    
    data['labels'] = data.pop('IsToxic')
    data['text'] = data.pop('Text')
    # Map true/false to 1/0
    data['labels'] = 1 if data['labels'] == True else 0
    return data

# Run the preprocessing
dataset = dataset.map(preprocess_data)

# Split the dataset into train-test
dict_train_test_split = dataset["train"].train_test_split(test_size=0.2, seed=823)

# Access the train and test sets
dict_train = dict_train_test_split["train"]
dict_test = dict_train_test_split["test"]

##### (1) Use the Pretrained Network directly

##### Initialize the Tokenizer and the Model

In [3]:
# Can also do this
# Initialize tokenizer and model
model = "martin-ha/toxic-comment-model"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForSequenceClassification.from_pretrained(model)  # Adjust num_labels as needed
classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer)

# Use pipeline(task="text-classification") for simplicity and ease of use, especially for standard tasks where you don't need much customization.
# Use TextClassificationPipeline when you need more control over the pipeline components or when you're working on a more complex or customized task.

c:\Users\djbac\anaconda3\envs\torch-gpu\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [21]:
inputs_test = dict_test['text']
actual_labels = dict_test["labels"]
# Make predictions
predictions = classifier(inputs_test)
# Predicted labels are "non-toxic" and "toxic", so we still need to map that
label_mapping = {"non-toxic": 0, "toxic": 1}
predicted_labels = [label_mapping[pred['label']] for pred in predictions]

In [23]:
# Print classification report
print(classification_report(actual_labels, predicted_labels))

              precision    recall  f1-score   support

           0       0.71      0.85      0.78       118
           1       0.70      0.51      0.59        82

    accuracy                           0.71       200
   macro avg       0.71      0.68      0.68       200
weighted avg       0.71      0.71      0.70       200




##### (2) Perform Finetuning using the dataset

In [6]:
# Tokenize the dataset
# Here is where you specify the input column
def tokenize_function(input_string):
    return tokenizer(input_string['text'], padding='max_length', truncation=True, truncation_strategy='longest_first')

# Apply the tokenization function to your dataset
dict_train_tokenized = dict_train.map(tokenize_function, batched=True)
dict_test_tokenized = dict_test.map(tokenize_function, batched=True)

# Format dataset for PyTorch
# Here is where you specify the label/target column
dict_train_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
dict_test_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [7]:
# Hyperparams + other config
num_train_epochs = 10
per_device_train_batch_size = 16
per_device_eval_batch_size = 32
weight_decay = 0.01
learning_rate = 2e-5
logging_steps = 10
warmup_steps = 50 # Calculated using total_train_steps = len(dict_train_tokenized) // per_device_train_batch_size*num_train_epochs

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',                                    # Directory to save results
    num_train_epochs=num_train_epochs,                         # Number of training epochs
    per_device_train_batch_size=per_device_train_batch_size,   # Batch size per device during training
    per_device_eval_batch_size=per_device_eval_batch_size,     # Batch size per device during evaluation
    warmup_steps=warmup_steps,                                 # Number of warmup steps for learning rate scheduler
    weight_decay=weight_decay,                                 # Strength of weight decay
    learning_rate=learning_rate,                               # Learning Rate
    logging_dir='./logs',                                      # Directory for storing logs
    logging_steps=logging_steps,                               # Frequency of logging steps
    evaluation_strategy="epoch",
    save_strategy="epoch",                                  
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

In [8]:
# Initialize the Trainer
trainer = Trainer(
    model=model,                          # The model to train
    args=training_args,                   # Training arguments
    train_dataset=dict_train_tokenized,   # The dataset to train on
)

In [9]:
# Start training
trainer.train()

ClearML Task: overwriting (reusing) task id=9e614809235d475da9b542269ae68b6b
2024-08-20 00:03:30,428 - clearml.Repository Detection - WARNING - Could not read Jupyter Notebook: No module named 'nbconvert'
2024-08-20 00:03:30,488 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/a280739eaca04ea6b7be4b98665e026b/experiments/9e614809235d475da9b542269ae68b6b/output/log
2024-08-20 00:03:41,656 - clearml.Task - WARNING - Parameters must be of builtin type (Transformers/accelerator_config[AcceleratorConfig])


  2%|▏         | 10/500 [00:31<26:29,  3.24s/it]

{'loss': 0.9185, 'grad_norm': 6.162717819213867, 'learning_rate': 4.0000000000000003e-07, 'epoch': 0.2}


  4%|▍         | 20/500 [01:04<25:55,  3.24s/it]

{'loss': 1.048, 'grad_norm': 81.02702331542969, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.4}


...

{'loss': 0.4915, 'grad_norm': 4.11275577545166, 'learning_rate': 0.0, 'epoch': 10.0}
2024-08-20 00:43:33,099 - clearml.storage - INFO - Starting upload: C:\Users\djbac\AppData\Local\Temp\model_package.mcwtobc7.zip => https://files.clear.ml/HuggingFace Transformers/Trainer.9e614809235d475da9b542269ae68b6b/models/checkpoint-500.zip


100%|██████████| 500/500 [39:39<00:00,  4.39s/it]

{'train_runtime': 2417.5998, 'train_samples_per_second': 3.309, 'train_steps_per_second': 0.207, 'train_loss': 0.6639747095108032, 'epoch': 10.0}


100%|██████████| 500/500 [1:24:20<00:00, 10.12s/it]


TrainOutput(global_step=500, training_loss=0.6639747095108032, metrics={'train_runtime': 2417.5998, 'train_samples_per_second': 3.309, 'train_steps_per_second': 0.207, 'total_flos': 1059739189248000.0, 'train_loss': 0.6639747095108032, 'epoch': 10.0})

In [10]:
# Evaluate the model on the test set
eval_results = trainer.evaluate(eval_dataset=dict_test_tokenized)
print(f"Evaluation results: {eval_results}")

100%|██████████| 7/7 [00:38<00:00,  5.54s/it]

Evaluation results: {'eval_loss': 0.6198015809059143, 'eval_runtime': 47.3297, 'eval_samples_per_second': 4.226, 'eval_steps_per_second': 0.148, 'epoch': 10.0}


In [11]:
# Make predictions on the test set
predictions, labels, _ = trainer.predict(test_dataset=dict_test_tokenized)
# Convert predictions to labels
predicted_labels = predictions.argmax(axis=-1)

100%|██████████| 7/7 [00:38<00:00,  5.48s/it]


In [12]:
from sklearn.metrics import classification_report
print(classification_report(labels, predicted_labels))

              precision    recall  f1-score   support

           0       0.84      0.66      0.74       118
           1       0.63      0.82      0.71        82

    accuracy                           0.72       200
   macro avg       0.73      0.74      0.72       200
weighted avg       0.75      0.72      0.73       200



In [197]:
predicted_labels

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 0], dtype=int64)

#### Load the Dataset

In [167]:
df = pd.read_csv("datasets\mtsamples.csv", index_col=0)

In [168]:
# Filtering to 12 classes for simplicity
list_med_sp = [
    " Allergy / Immunology",
    " Bariatrics",
    " Cardiovascular / Pulmonary",
    " Urology",
    " Dentistry",
    " Rheumatology",
    " Radiology",
    " Psychiatry / Psychology",
    " Podiatry",
    " Orthopedic",
    " Opthalmology",
    " Neurology"
]
# Filter the DataFrame
df = df[df['medical_specialty'].isin(list_med_sp)]

# df_stratified, _ = train_test_split(df, train_size=0.03, stratify=df['medical_specialty'])

#### Specify the candidate labels for the ZSC

In [169]:
candidate_labels = df["medical_specialty"].unique().tolist()

#### Create the Classifier

In [170]:
classifier = (
    pipeline(task="zero-shot-classification",
             model="tasksource/deberta-small-long-nli"
    )
)

In [171]:
# Function to classify a single description
def classify_description(description):
    result = classifier(description, candidate_labels)
    # Find the index of the maximum score
    max_index = result['scores'].index(max(result['scores']))
    # Return the label with the highest score
    return result['labels'][max_index], result['scores'][max_index]

#### Zero-shot

In [172]:
df[['zero_shot_class', 'score']] = (
    df['description']
    .swifter.progress_bar(enable=True)
    .apply(classify_description)  # Apply function directly
    .apply(pd.Series)  # Convert tuple/list output into DataFrame
)

Pandas Apply: 100%|██████████| 1543/1543 [49:52<00:00,  1.94s/it]


In [174]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Assuming 'y_true' is the actual label and 'y_pred' is the predicted label
y_true = df['medical_specialty']
y_pred = df['zero_shot_class']

# Get accuracy score
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Get classification report
print("Classification Report:")
print(classification_report(y_true, y_pred))

# Get confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Accuracy: 0.53
Classification Report:
                             precision    recall  f1-score   support

       Allergy / Immunology       0.16      1.00      0.27         7
                 Bariatrics       0.75      0.83      0.79        18
 Cardiovascular / Pulmonary       0.81      0.23      0.36       372
                  Dentistry       0.76      0.81      0.79        27
                  Neurology       0.71      0.31      0.44       223
                 Orthopedic       0.71      0.74      0.72       355
                   Podiatry       0.55      0.38      0.45        47
    Psychiatry / Psychology       0.78      0.74      0.76        53
                  Radiology       0.31      0.76      0.44       273
               Rheumatology       0.18      0.40      0.25        10
                    Urology       0.93      0.59      0.73       158

                   accuracy                           0.53      1543
                  macro avg       0.60      0.62      0.54     